### Finite state automata

We will cover the following topics in this session:
- Finite state automata: definition, implementation
- Regular languages: definition, examples
- Regular languages
- Recongizing regular languages with automata

#### Finite state machines

A finite state machine is a simple model of computation,
with the following components:
- A set of machine states $S$.
- A start state $S_0 \in S$.
- A set of accepting states $F \subseteq S$.
- A set of input characters/alphabet $\Sigma$.
- A transition function $\delta : S \times \Sigma \to S$.

Here, $\delta$ describes the machine's action at each given input: $\delta(s, x) = s'$ means that
the machine at state $s$ steps to state $s'$ if the input is $x$.

In OCaml terms, we need:
```
type states
val start : state
val accept : state -> bool
val alphabet : char -> bool
val transit : state -> char -> state
```

Here is a simple example of a (deterministic) finite state machine.
- States $S = \{A, B\}$
- Start state $S_0 = A$
- Accepting states $F = \{ A \}$
- Alphabet $\Sigma = \{a, b\}$
- Transition $\delta$:
    - $\delta(A, a) = A$
    - $\delta(A, b) = B$
    - $\delta(B, a) = A$
    - $\delta(B, b) = B$

In [ ]:
type states = A | B

let start = A

let accept = function
	| A -> false
	| B -> true

let alphabet = function
	| 'a' | 'b' -> true
	| _ -> false

let transit s c = 
	if not (alphabet c) then raise (Invalid_argument "Unexpected character!")
	else
		match (s, c) with
			| A, 'a' -> A
			| A, 'b' -> B
			| B, 'a' -> A
			| B, 'b' -> B
			| _ -> assert false

Given a list of characters (a string), we can 
feed it into the machine, evaluating each step with
the transit function.
If the machine ends up in an accepting state when
it has consumed all input characters, we say that 
this machine *accepts* the input string.

What kind of strings are accepted by the machine above?

In [ ]:
let match_str str = 
    (* Turning string into list of characters *)
	let len = String.length str in
	let ls = List.init len (fun i -> String.get str i) in
    (* Running the machine *)
	let rec matching state = function
		| [] -> accept state
		| c :: cs -> 
			let state' = transit state c in
			matching state' cs
	in

		matching start ls

let _ = match_str "abababba"

Here is a more complicated machine. Again, what kind of strings are accepted by it?

In [ ]:
type states = A0 | A1 | A2 | Acc 

let start = A0

let accept = function
	| Acc -> true
	| _ -> false

let alphabet = function
	| 'a' | 'b' -> true
	| _ -> false

let transit s c = 
	if not (alphabet c) then raise (Invalid_argument "Unexpected character!")
	else
		 match (s, c) with
			| A0, 'a' -> A1
			| A1, 'a' -> A2
			| A2, 'a' -> Acc
			| Acc, 'a' -> Acc
			| state, 'b' -> state
			| _ -> assert false

let match_str str = 
    (* Turning string into list of characters *)
	let len = String.length str in
	let ls = List.init len (fun i -> String.get str i) in
    (* Running the machine *)
	let rec matching state = function
		| [] -> accept state
		| c :: cs -> 
			let state' = transit state c in
			matching state' cs
	in

		matching start ls

let _ = match_str "abababba"

#### Application: ATM machine

We now see a simple application of using a finite automaton to describe the internal states of an ATM machine. The machine should have three states: 
- `Ready`, it is ready to use.
- `CardInserted`, the user has inserted a bank card, but the password is not verified.
- `LoggedIn`, the password is verified and the user can cash out.

We also have the following actions available: 
- `InsertCard`, only valid if the machine is `Ready`, turn the machine into `CardInserted` state.
- `InputPassword`, only valid in `CardInserted` state, transits to `LoggedIn` if the password is correct, no transition otherwise. 
- `CashOut`, only valid in `LoggedIn` state.
- `LogOut`, sets the machine to `Ready` state, return the bank card if one is inserted.

The above description can be easily encoded as the following finite automaton:
the states are the machine states, and the characters in the alphabet are the actions.
We can then run the machine with a list of given actions, and define an `execute_trace` function to 
trace the machine states that appeared during execution.

In [13]:
type states = Ready | CardInserted | LoggedIn
type actions = InsertCard | InputPassword of string | CashOut | LogOut

let start = Ready
let password = "password"

let transit s c = match (s, c) with
    | Ready, InsertCard -> CardInserted
    | CardInserted, InputPassword str -> 
        if str = password then LoggedIn
        else CardInserted
    | CardInserted, LogOut -> Ready
    | LoggedIn, LogOut -> Ready
    | LoggedIn, CashOut -> LoggedIn
    | s, _ -> s

(* execute : state -> actions list -> state *)
let rec execute s = function
    | [] -> s
    | act :: actions -> execute (transit s act) actions

(* execute_trace: state -> actions list -> state list *)
let execute_trace state actions =
    let rec aux s acc = function
        | [] -> List.rev (s :: acc)
        | act :: actions -> aux (transit s act) (s :: acc) actions
    in
        aux state [] actions

type states = Ready | CardInserted | LoggedIn


type actions = InsertCard | InputPassword of string | CashOut | LogOut


val start : states = Ready


val password : string = "password"


val transit : states -> actions -> states = <fun>


val execute : states -> actions list -> states = <fun>


val execute_trace : states -> actions list -> states list = <fun>


val actions1 : actions list =
  [InsertCard; InputPassword "password"; CashOut; CashOut; LogOut]


val actions2 : actions list =
  [InsertCard; LogOut; InputPassword "password"; InsertCard;
   InputPassword "123"; CashOut]


- : states list = [Ready; CardInserted; LoggedIn; LoggedIn; LoggedIn; Ready]


In [14]:
let actions1 = [InsertCard; InputPassword "password"; CashOut; CashOut; LogOut]
let actions2 = [InsertCard; LogOut; InputPassword "password"; InsertCard; InputPassword "123"; CashOut;]

let _ = execute_trace start actions1
let _ = execute_trace start actions2

val actions1 : actions list =
  [InsertCard; InputPassword "password"; CashOut; CashOut; LogOut]


val actions2 : actions list =
  [InsertCard; LogOut; InputPassword "password"; InsertCard;
   InputPassword "123"; CashOut]


- : states list = [Ready; CardInserted; LoggedIn; LoggedIn; LoggedIn; Ready]


- : states list =
[Ready; CardInserted; Ready; Ready; CardInserted; CardInserted; CardInserted]
